In [2]:
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup as bs
import requests
import time
from geopy import geocoders

# Import API key
from api_keys import api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

C:\Users\nicho\Anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
# List of each week number
weeks = list(range(1,18))
# Api Url
base_url = "http://api.fantasy.nfl.com/v1/players/stats?statType=weekStats&season=2018&week={}&form=json"

In [4]:
# Empty dataframe to contain all the api data
temp_final_df = pd.DataFrame()

In [ ]:
# Makes an api call for each week's data and appends it to main dataframe
for week in weeks:
    target_url = base_url.format(week)
    temp = requests.get(target_url).json()['players']
    temp_df = pd.DataFrame(temp)
    temp_df = temp_df.drop(columns = 'stats')
    temp_df['week'] = week
    temp_final_df = temp_final_df.append(temp_df)

In [ ]:
temp_final_df[temp_final_df['week'] == 1]
team_names = temp_final_df.teamAbbr.unique()
team_names

In [ ]:
data = pd.read_csv('2018_Schedule_City.csv')

In [ ]:
data['away_abrev'] = data['Away'].str[0:4]
data['home_abrev'] = data['Home'].str[0:4]


In [ ]:
away_abrev = []

for awy in data['away_abrev']:
    away_abrev.append(process.extract(awy,team_names)[0][0])

In [ ]:
home_abrev = []

for hme in data['home_abrev']:
    home_abrev.append(process.extract(hme,team_names)[0][0])

In [ ]:
data['away_abrev'] = away_abrev
data['home_abrev'] = home_abrev
data

# STOP

In [ ]:
import time
from datetime import datetime
date_str2 = '6/6/18'
date_dt2 = datetime.strptime(date_str2, '%m/%d/%y')
date_dt2

In [42]:
city_list = data['City'].tolist()
city_weather = {}
loc = geolocator.geocode(city_list[0])
loc.address

'Philadelphia, Philadelphia County, Pennsylvania, USA'

In [22]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent='myapplication')
location = geolocator.geocode(data['City'][0])
print(location.address)
print(geolocator.geocode(data['City'][0]).address)

Philadelphia, Philadelphia County, Pennsylvania, USA
Philadelphia, Philadelphia County, Pennsylvania, USA


for city in city_list:
    loc = geolocator.geocode(city)
    city_weather[city] = geolocator.geocode(loc.address)

city_weather

In [ ]:
data['City'][0]

In [ ]:
# Base url used to access the openweathermap api 
url = "http://api.openweathermap.org/data/2.5/weather?"
# Url used to actually access the api data using the provided api key
query_url = f"{url}appid={api_key}&units=imperial&q="

In [ ]:
response = requests.get(query_url + location.address)

In [ ]:
print(response)

response.json()

In [ ]:
# Base url used to access the openweathermap api 
url = "http://api.openweathermap.org/data/2.5/weather?"
# Url used to actually access the api data using the provided api key
query_url = f"{url}appid={api_key}&units=imperial&q="

# Counter used to count the amount of records being called from openweathermap's api
rec_count = 0
# Empty list to store the each city's weather data
city_data = []

print("Beginning Data Retrieval")    
print("-----------------------------")
# Loop through the list of cities and perform a request for data on each
for city in cities:
    response = requests.get(query_url + city)
    
    # If statement used to determine wether the pull was successful and add that to the city_data list
    if str(response) == "<Response [200]>":
        rec_count +=1
        print(f"Processing Record {rec_count} of Set 1 | {city}")
        city_data.append(response.json())
    
    # Prints if there is no data on that city and leaves out the city name
    else:
        print("City not found. Skipping...")